In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings

PATH = "data/"

In [2]:
pd.set_option("display.max_columns",None)

In [3]:
train = pd.read_csv(PATH+"train_0202.csv")
test = pd.read_csv(PATH+"test_0202.csv")

In [4]:
print(train.columns)
print(test.columns)

Index(['user_id', 'model_nm', 'update', 'err_1', 'err_2', 'err_3', 'err_4',
       'err_5', 'err_6', 'err_7', 'err_8', 'err_9', 'err_10', 'err_11',
       'err_12', 'err_13', 'err_14', 'err_15', 'err_16', 'err_17', 'err_18',
       'err_19', 'err_20', 'err_21', 'err_22', 'err_23', 'err_24', 'err_25',
       'err_26', 'err_27', 'err_28', 'err_30', 'err_31', 'err_32', 'err_33',
       'err_34', 'err_35', 'err_36', 'err_37', 'err_38', 'err_39', 'err_40',
       'err_41', 'err_42', 'fw_rank', 'q_meanmax_0', 'q_meanmax_1',
       'q_meanmax_2', 'q_meanmax_5', 'q_meanmax_6', 'q_meanmax_7',
       'q_meanmax_8', 'q_meanmax_9', 'q_meanmax_10', 'q_meanmax_11',
       'q_meanmax_12', 'q_chg_0', 'q_chg_1', 'q_chg_2', 'q_chg_5', 'q_chg_6',
       'q_chg_11', 'q1_nonzero_counts', 'q2_nonzero_counts',
       'q5_nonzero_counts', 'q6_nonzero_counts', 'q11_nonzero_counts',
       'have_quality', 'label', 'weekend_error', 'sunday_error', 'night_error',
       'max_err_day', 'max_err_sunday', 'max_err_w

In [5]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = train.drop(columns = ["user_id", "update", "label", "err_8", "err_9"])
train_y = train.label
print(train_x.shape)
print(train_y.shape)

(15000, 75)
(15000,)


In [6]:
train_x

,model_nm,err_1,err_2,err_3,err_4,err_5,err_6,err_7,err_10,err_11,err_12,err_13,err_14,err_15,err_16,err_17,err_18,err_19,err_20,err_21,err_22,err_23,err_24,err_25,err_26,err_27,err_28,err_30,err_31,err_32,err_33,err_34,err_35,err_36,err_37,err_38,err_39,err_40,err_41,err_42,fw_rank,q_meanmax_0,q_meanmax_1,q_meanmax_2,q_meanmax_5,q_meanmax_6,q_meanmax_7,q_meanmax_8,q_meanmax_9,q_meanmax_10,q_meanmax_11,q_meanmax_12,q_chg_0,q_chg_1,q_chg_2,q_chg_5,q_chg_6,q_chg_11,q1_nonzero_counts,q2_nonzero_counts,q5_nonzero_counts,q6_nonzero_counts,q11_nonzero_counts,have_quality,weekend_error,sunday_error,night_error,max_err_day,max_err_sunday,max_err_weekend,second_max_err,third_max_err,night_max_err,most_used_model,fwver_changed
0,1,0.0,0.0,8.0,104.0,0.0,1.0,1.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.0,8.000000,0.000000,0.0,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.0,2.0,106.0,52.0,221.0,20.0,16.0,16.0,19.0,18.0,16.0,3,1
1,1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,0.967949,3.674242,0.385522,3.674242,1.121633,0.492424,0.287879,0.030303,0.0,9.707071,0.367424,0.0,1.318182,1.444444,1.318182,4.631313,1.702020,1.318182,0.030303,0.000000,3.792929,0.065657,0.0,0.0,256.0,139.0,1015.0,1452.0,62.0,62.0,62.0,58.0,373.0,2,2
2,1,0.0,0.0,2.0,132.0,1.0,2.0,1.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,1.666667,0.166667,1.666667,0.666667,3.666667,44.000000,0.000000,1.0,8.000000,0.166667,0.0,2.000000,1.000000,2.000000,12.000000,3.000000,1.000000,0.000000,1.000000,12.000000,3.000000,0.0,8.0,100.0,58.0,200.0,17.0,16.0,16.0,16.0,16.0,15.0,3,1
3,1,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,0.947059,2.512626,0.261785,2.512626,0.982744,0.405724,0.257576,0.000000,0.0,8.459596,0.250842,0.0,0.717172,0.787879,0.717172,3.323232,0.994949,0.712121,0.000000,0.000000,3.030303,0.050505,0.0,0.0,79.0,57.0,265.0,24.0,21.0,21.0,21.0,21.0,21.0,2,2
4,1,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,0.833977,1.666667,0.166667,1.666667,0.416667,7.250000,87.000000,0.000000,0.0,3.000000,0.166667,0.0,1.000000,1.000000,1.000000,3.000000,2.000000,1.000000,0.000000,0.000000,3.000000,1.000000,0.0,2.0,192.0,108.0,528.0,102.0,29.0,29.0,54.0,49.0,38.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,1.000000,6.666667,0.666667,6.666667,0.833333,0.666667,0.000000,0.000000,0.0,3.000000,0.666667,0.0,2.000000,2.000000,2.000000,4.000000,2.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.0,2.0,73.0,34.0,127.0,30.0,17.0,22.0,24.0,24.0,21.0,2,1
14996,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833333,0.433502,0.183081,0.496633,2.534933,0.526515,3.358586,0.000000,0.0,28.388889,0.178451,0.0,0.212121,0.651515,0.242424,5.505051,1.121212,0.626263,0.000000,0.737374,5.050505,0.297980,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,3,1
14997,1,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,16.0,18.0,4

In [23]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.55
# 파라미터 설정
params =      {
                'boosting_type' : 'dart',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'feature_fraction' : 0.7,
                'bagging_fraction': 0.9, 
                'bagging_freq': 5,
                'seed': 34,
                'learning_rate' : 0.01,
                "categorical_feature" : None,
                "labmda" : 0.2
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=34)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x.iloc[train_idx,:]
    y = train_y[train_idx]
    valid_x = train_x.iloc[val_idx,:]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 2000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 300,
    )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1075: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  warnings.warn('{0} keyword has been found in `params` and will be ignored.\n'


[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Info] Number of positive: 4004, number of negative: 7996
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003397 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11298
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 75
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333667 -> initscore=-0.691648
[LightGBM] [Info] Start training from score -0.691648
[300]	valid_0's auc: 0.825362	valid_0's pr_auc: 0.743253
[600]	valid_0's auc: 0.828055	valid_0's pr_auc: 0.749649
[900]	valid_0's auc: 0.831955	valid_0's pr_auc: 0.756385
[1200]	valid_0's auc: 0.83377	valid_0's pr_auc: 0.758887
[1500]	valid_0's auc: 0.835537	valid_0's pr_auc: 0.761347
[1800]	valid_0's auc: 0.83638	valid_0's pr_auc: 0.761816
[LightGBM] [Warning]

/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1075: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  warnings.warn('{0} keyword has been found in `params` and will be ignored.\n'


[300]	valid_0's auc: 0.818021	valid_0's pr_auc: 0.732605
[600]	valid_0's auc: 0.819914	valid_0's pr_auc: 0.737104
[900]	valid_0's auc: 0.820768	valid_0's pr_auc: 0.736943
[1200]	valid_0's auc: 0.823016	valid_0's pr_auc: 0.737465
[1500]	valid_0's auc: 0.823501	valid_0's pr_auc: 0.738807
[1800]	valid_0's auc: 0.823253	valid_0's pr_auc: 0.738328
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Info] Number of positive: 4002, number of negative: 7998
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005475 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11281
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 75
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333500 -> initscore=-0.692397
[LightGBM] [Info] Start training from score -0.692397


/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1075: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  warnings.warn('{0} keyword has been found in `params` and will be ignored.\n'


[300]	valid_0's auc: 0.804058	valid_0's pr_auc: 0.719786
[600]	valid_0's auc: 0.806723	valid_0's pr_auc: 0.719215
[900]	valid_0's auc: 0.809719	valid_0's pr_auc: 0.723038
[1200]	valid_0's auc: 0.810676	valid_0's pr_auc: 0.722488
[1500]	valid_0's auc: 0.811168	valid_0's pr_auc: 0.723669
[1800]	valid_0's auc: 0.811482	valid_0's pr_auc: 0.723873
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Info] Number of positive: 4004, number of negative: 7996
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003470 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11286
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 75
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333667 -> initscore=-0.691648
[LightGBM] [Info] Start training from score -0.691648


/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1075: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  warnings.warn('{0} keyword has been found in `params` and will be ignored.\n'


[300]	valid_0's auc: 0.81106	valid_0's pr_auc: 0.746928
[600]	valid_0's auc: 0.814575	valid_0's pr_auc: 0.751629
[900]	valid_0's auc: 0.818288	valid_0's pr_auc: 0.756494
[1200]	valid_0's auc: 0.819856	valid_0's pr_auc: 0.756849
[1500]	valid_0's auc: 0.821111	valid_0's pr_auc: 0.757333
[1800]	valid_0's auc: 0.821697	valid_0's pr_auc: 0.757584
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Info] Number of positive: 3977, number of negative: 8023
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003226 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11298
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 75
[LightGBM] [Warning] Unknown parameter: labmda
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.331417 -> initscore=-0.701785
[LightGBM] [Info] Start training from score -0.701785


/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1075: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  warnings.warn('{0} keyword has been found in `params` and will be ignored.\n'


[300]	valid_0's auc: 0.82057	valid_0's pr_auc: 0.747676
[600]	valid_0's auc: 0.822084	valid_0's pr_auc: 0.749317
[900]	valid_0's auc: 0.824134	valid_0's pr_auc: 0.752419
[1200]	valid_0's auc: 0.825174	valid_0's pr_auc: 0.755009
[1500]	valid_0's auc: 0.825353	valid_0's pr_auc: 0.754895
[1800]	valid_0's auc: 0.825786	valid_0's pr_auc: 0.755632


In [24]:
auc_scores

[0.8363571551675765,
 0.8234640993622507,
 0.8115586817991627,
 0.8218823397381942,
 0.825050149050345]

In [25]:
print(np.mean(auc_scores))

0.823662485023506


In [250]:
test.columns

Index(['Unnamed: 0', 'user_id', 'model_nm', 'update', 'err_1', 'err_2',
       'err_3', 'err_4', 'err_5', 'err_6', 'err_7', 'err_8', 'err_9', 'err_10',
       'err_11', 'err_12', 'err_13', 'err_14', 'err_15', 'err_16', 'err_17',
       'err_18', 'err_19', 'err_20', 'err_21', 'err_22', 'err_23', 'err_24',
       'err_25', 'err_26', 'err_27', 'err_28', 'err_30', 'err_31', 'err_32',
       'err_33', 'err_34', 'err_35', 'err_36', 'err_37', 'err_38', 'err_39',
       'err_40', 'err_41', 'err_42', 'fw_rank', 'q_meanmax_0', 'q_meanmax_1',
       'q_meanmax_2', 'q_meanmax_5', 'q_meanmax_6', 'q_meanmax_7',
       'q_meanmax_8', 'q_meanmax_9', 'q_meanmax_10', 'q_meanmax_11',
       'q_meanmax_12', 'q_chg_0', 'q_chg_1', 'q_chg_2', 'q_chg_5', 'q_chg_6',
       'q_chg_11', 'q1_nonzero_counts', 'q2_nonzero_counts',
       'q5_nonzero_counts', 'q6_nonzero_counts', 'q11_nonzero_counts',
       'have_quality', 'weekend_error', 'sunday_error', 'night_error',
       'max_err_day', 'max_err_sunday', 'max_

In [172]:
test_x = test.drop(columns = ["Unnamed: 0","user_id", "update", "err_8", "err_9"])

In [173]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [174]:
pred_ensemble

array([[0.9209025 ],
       [0.19292342],
       [0.32233817],
       ...,
       [0.57412663],
       [0.90484854],
       [0.36477591]])

In [175]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [176]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [177]:
sample_submssion.to_csv(PATH+"0202(4)-0.82449.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.920903
1,30001,0.192923
2,30002,0.322338
3,30003,0.743424
4,30004,0.907082
...,...,...
14994,44994,0.310648
14995,44995,0.238695
14996,44996,0.574127
14997,44997,0.904849


In [134]:
best = pd.read_csv(PATH + "0202.csv")

In [135]:
best["mine2"] = pred_ensemble

In [136]:
best

,user_id,problem,mine2
0,30000,0.925746,0.913719
1,30001,0.179326,0.196820
2,30002,0.329626,0.324847
3,30003,0.745617,0.765818
4,30004,0.898933,0.902437
...,...,...,...
14994,44994,0.295214,0.304689
14995,44995,0.261761,0.235081
14996,44996,0.584120,0.591373
14997,44997,0.888072,0.898558


In [137]:
best.corr()

,user_id,problem,mine2
user_id,1.000000,-0.006602,-0.007114
problem,-0.006602,1.000000,0.997967
mine2,-0.007114,0.997967,1.000000


In [1]:
import numpy as np
import pandas as pd
import os
from scipy import stats
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn import linear_model
import datetime

# K Fold 및 기본 데이터 준비
folds = KFold(n_splits = 5, shuffle = True, random_state = 511)
oof_preds = np.zeros(train.shape[0])
# catboost 이용

NameError: name 'train' is not defined

In [ ]:
print("using catboost")
sub_preds_catboost = np.zeros(test.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_1)) :
trn_x, trn_y = x_train.ix[trn_idx], y_train[trn_idx]
val_x, val_y = x_train.ix[val_idx], y_train[val_idx]
cb_model = CatBoostRegressor(iterations=1000,
learning_rate=0.1,
depth=4,
l2_leaf_reg=20,
bootstrap_type='Bernoulli',
subsample=0.6,
eval_metric='RMSE',
metric_period=50,
od_type='Iter',
od_wait=45,
random_seed=17,
allow_writing_files=False)
cb_model.fit(trn_x, trn_y,
eval_set=(val_x, val_y),
cat_features=[],
use_best_model=True,
verbose=True)
oof_preds_catboost[val_idx] = cb_model.predict(val_x)
sub_preds_catboost += cb_model.predict(x_test) / folds.n_splits
sub_cat = pd.DataFrame()
sub_cat['ID'] = test_1['ID']
sub_cat['target'] = np.exp(sub_preds_catboost)-1
sub_cat.to_csv('sansander_catboost.csv', index = False)
# lgm 이용
print("lightgbm")
sub_preds_lgm = np.zeros(test.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_1)) :
trn_x, trn_y = x_train.ix[trn_idx], y_train[trn_idx]
val_x, val_y = x_train.ix[val_idx], y_train[val_idx]
params = {'learning_rate': 0.01,
'max_depth': 16,
'boosting': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'is_training_metric': True,
'num_leaves': 144,
'feature_fraction': 0.9,
'bagging_fraction': 0.7,
'bagging_freq': 5,
'seed':2018}
train_T = lgb.Dataset(trn_x, label=trn_y)
val_T = lgb.Dataset(val_x, label=val_y)
model1 = lgb.train(params, train_T, 1000, val_T, verbose_eval=100, early_stopping_rounds=100)
sub_preds_lgm += model1.predict(x_test) / folds.n_splits
oof_preds_lgm[val_idx] = model1.predict(val_x)
sub_lgm = pd.DataFrame()
sub_lgm['ID'] = test_1['ID']
sub_lgm['target'] = np.exp(sub_preds_lgm)-1
sub_lgm.to_csv('sansander_lgb.csv', index = False)
# xgboost준비
print("xgboost")
sub_preds_xgb = np.zeros(test.shape[0])
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train_1)) :
trn_x, trn_y = x_train.ix[trn_idx], y_train[trn_idx]
val_x, val_y = x_train.ix[val_idx], y_train[val_idx]
params = {'objective': 'reg:linear',
'eval_metric': 'rmse',
'eta': 0.005,
'max_depth': 15,
'subsample': 0.6,
'colsample_bytree': 0.6,
'alpha':0.001,
'random_state': 42,
'silent': True}
tr_data = xgb.DMatrix(trn_x, trn_y)
va_data = xgb.DMatrix(val_x, val_y)
watchlist = [(tr_data, 'train'), (va_data, 'valid')]
model_xgb = xgb.train(params, tr_data, 2100, watchlist, maximize=False, early_stopping_rounds = 100, verbose_eval=100)
test_data = xgb.DMatrix(x_test)
sub_preds_xgb += model_xgb.predict(test_data) / folds.n_splits
oof_preds_xgb[val_idx] = model_xgb.predict(va_data)
sub_xgb = pd.DataFrame()
sub_xgb['ID'] = test_1['ID']
sub_xgb['target'] = np.exp(sub_preds_xgb)-1
sub_xgb.to_csv('sansander_xgb.csv', index = False)
# stacking 준비
regr = linear_model.LinearRegression(fit_intercept = False)
stackedmodel = regr.fit(a[['xgb','lgm','catboost']], a[['y']])
print(stackedmodel)
# catboost는 일단 빼고 가중치 계산
result_raw = pd.DataFrame(data = {'ID':test_1['ID'].values, 'xgb' : np.expm1(sub_preds_xgb), 'lgm' : np.expm1(sub_preds_lgm)})
result_raw['result1'] = result_raw.apply(lambda row: row.xgb * 0.45] + row.lgm * 0.55, axis=1)
# target을 구하기 위해 join을 하고, leak된 행만 바꾸고 저장.
leaked_join_1 = result_raw.merge(leak_id, left_on='ID', right_on='id', how='outer')
leaked_join_1['target_final'] = leaked_join_1.apply(lambda row: row.target if row.boolean == True else row.result1, axis=1)
submission_stacked = c[['ID', 'target_final']]
submission_stacked.columns = ['ID', 'target']
submission_stacked.to_csv('stacking' + str(datetime.datetime.now()) + '.csv', index = False)


출처: https://gorakgarak.tistory.com/1278 [먹거리 만드는 열정맨 고락가락]